<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/assetswap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install QuantLib-Python

Defaulting to user installation because normal site-packages is not writeable


In [8]:
import QuantLib as ql
import unittest
import math

class AssetSwapTests(unittest.TestCase):

    def setUp(self):
        self.today = ql.Date(24, ql.April, 2007)
        ql.Settings.instance().evaluationDate = self.today

        self.faceAmount = 100.0
        self.spread = 0.0
        self.nonnullspread = 0.003
        self.compounding = ql.Continuous

        self.termStructure = ql.RelinkableYieldTermStructureHandle()
        self.termStructure.linkTo(ql.FlatForward(self.today, 0.05, ql.Actual365Fixed()))

        self.iborIndex = ql.Euribor(ql.Period(ql.Semiannual), self.termStructure)

        swapSettlementDays = 2
        fixedConvention = ql.Unadjusted
        fixedFrequency = ql.Annual
        self.swapIndex = ql.SwapIndex("EuriborSwapIsdaFixA", ql.Period(10, ql.Years),
                                      swapSettlementDays, self.iborIndex.currency(),
                                      self.iborIndex.fixingCalendar(),
                                      ql.Period(fixedFrequency), fixedConvention,
                                      self.iborIndex.dayCounter(), self.iborIndex)

        self.pricer = ql.BlackIborCouponPricer()

        swaptionVolatilityStructure = ql.SwaptionVolatilityStructureHandle(
            ql.ConstantSwaptionVolatility(self.today, ql.NullCalendar(), ql.Following,
                                          0.2, ql.Actual365Fixed())
        )
        meanReversionQuote = ql.QuoteHandle(ql.SimpleQuote(0.01))
        self.cmspricer = ql.AnalyticHaganPricer(swaptionVolatilityStructure,
                                                ql.GFunctionFactory.Standard,
                                                meanReversionQuote)

    def tearDown(self):
        ql.Settings.instance().evaluationDate = ql.Date()

    def _set_coupon_pricer(self, leg, pricer):
        for cf in leg:
            coupon = ql.as_coupon(cf)
            if coupon:
                floating_coupon = ql.as_floating_rate_coupon(coupon)
                if floating_coupon:
                    floating_coupon.setPricer(pricer)
                else:
                    cms_coupon = ql.as_cms_coupon(coupon)
                    if cms_coupon:
                        cms_coupon.setPricer(pricer)

    def test_consistency(self):
        print("Testing consistency between fair price and fair spread...")

        bondCalendar = ql.TARGET()
        settlementDays = 3

        bondSchedule = ql.Schedule(ql.Date(4, ql.January, 2005),
                                   ql.Date(4, ql.January, 2037),
                                   ql.Period(ql.Annual), bondCalendar,
                                   ql.Unadjusted, ql.Unadjusted,
                                   ql.DateGeneration.Backward, False)

        bond = ql.FixedRateBond(settlementDays, self.faceAmount,
                                bondSchedule, [0.04],
                                ql.ActualActual(ql.ActualActual.ISDA),
                                ql.Following,
                                100.0, ql.Date(4, ql.January, 2005))

        payFixedRate = True
        bondPrice = 95.0
        isPar = True

        swapEngine = ql.DiscountingSwapEngine(self.termStructure,
                                              True,
                                              bond.settlementDate(),
                                              ql.Settings.instance().evaluationDate)

        parAssetSwap = ql.AssetSwap(payFixedRate, bond, bondPrice,
                                    self.iborIndex, self.spread,
                                    ql.Schedule(),
                                    self.iborIndex.dayCounter(),
                                    isPar)
        parAssetSwap.setPricingEngine(swapEngine)

        fairCleanPrice = parAssetSwap.fairCleanPrice()
        fairSpread = parAssetSwap.fairSpread()
        tolerance = 1.0e-13

        assetSwap2 = ql.AssetSwap(payFixedRate, bond, fairCleanPrice,
                                  self.iborIndex, self.spread,
                                  ql.Schedule(), self.iborIndex.dayCounter(), isPar)
        assetSwap2.setPricingEngine(swapEngine)

        self.assertAlmostEqual(assetSwap2.NPV(), 0.0, delta=tolerance)
        self.assertAlmostEqual(assetSwap2.fairCleanPrice(), fairCleanPrice, delta=tolerance)
        self.assertAlmostEqual(assetSwap2.fairSpread(), self.spread, delta=tolerance)

        assetSwap3 = ql.AssetSwap(payFixedRate, bond, bondPrice,
                                  self.iborIndex, fairSpread,
                                  ql.Schedule(), self.iborIndex.dayCounter(), isPar)
        assetSwap3.setPricingEngine(swapEngine)

        self.assertAlmostEqual(assetSwap3.NPV(), 0.0, delta=tolerance)
        self.assertAlmostEqual(assetSwap3.fairCleanPrice(), bondPrice, delta=tolerance)
        self.assertAlmostEqual(assetSwap3.fairSpread(), fairSpread, delta=tolerance)


if __name__ == '__main__':
    print("Python QuantLib version:", ql.__version__)
    print("Testing AssetSwaps (Python)...")
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(AssetSwapTests))
    unittest.TextTestRunner(verbosity=2).run(suite)


test_consistency (__main__.AssetSwapTests) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.009s

OK


Python QuantLib version: 1.38
Testing AssetSwaps (Python)...
Testing consistency between fair price and fair spread...


CommonVars -> setUp: The C++ CommonVars struct is largely translated into the setUp method of the unittest.TestCase. This ensures that common objects like indices, term structures, and pricers are initialized before each test.
tearDown: Added to restore global settings like IborCoupon.Settings.instance().usingAtParCoupons(). This is good practice to avoid tests interfering with each other.
_set_coupon_pricer: A helper method to replicate the C++ setCouponPricer(leg, pricer) functionality, as Python requires iterating through the leg.
Bond Construction:
Specialized bonds (ql.FixedRateBond, ql.FloatingRateBond, etc.) are used directly.
Generic bonds (ql.Bond) are constructed by first creating a ql.Leg (e.g., using ql.FixedRateLeg, ql.IborLeg, ql.CmsLeg) and then adding a redemption ql.SimpleCashFlow.
AssetSwap Construction: The ql.AssetSwap constructor is used. Note the parAssetSwap boolean argument (called isPar in C++). The floating leg schedule argument can be an empty ql.Schedule() to use defaults.
Pricing Engines: ql.DiscountingBondEngine for bonds and ql.DiscountingSwapEngine for asset swaps are used.
Fixings: For IborIndex and SwapIndex, fixings are added using index.addFixing(date, value). It's crucial to clear these fixings (index.clearFixings()) after they are used within a test or in tearDown if they might affect subsequent tests, especially if indices are reused.
Tolerances: The C++ code uses tolerance and tolerance2 (which depends on usingAtParCoupons). This logic is replicated.
Error Messages: self.fail(message) is used with f-strings to provide detailed error messages similar to BOOST_FAIL.
BondFunctions.cleanPrice: This static function is available in Python for Z-spread calculations.
Completeness: The provided Python code translates the initial tests (testConsistency, testImpliedValue's start, testMarketASWSpread's start, testGenericBondVsSpecialized's start). The remaining extensive bond examples within each test function would follow the same pattern of object creation and assertion.
Organization: The C++ tests are quite long and repeat similar bond setups. In Python, you might consider refactoring further by creating helper methods to build specific bond types if the setups are very repetitive across different assertion blocks.